# Description

Notebook to compute metrics.

# Setup

In [1]:
%matplotlib notebook
%load_ext autoreload

## Imports

In [2]:
# os related
from pathlib import Path
import os 

# data related
import pandas as pd
import missingno as msno
import datetime as dt
import numpy as np


# visual related
import matplotlib.pyplot as plt
# time related

from fonctions import *

## Paths

In [3]:

# MUST BE IN CODE FOLDER TO WORK !
PROJECT_PATH = Path(os.getcwd()).parent

RAW_DATA = PROJECT_PATH / "RAW_DATA" 
PROC_DATA = PROJECT_PATH / "PROC_DATA"

# RAW FILES
CSV_FILE = RAW_DATA / "Individual_stock_data_csv.csv"
XLSX_FILE = RAW_DATA / "Individual_stock_data.xlsx"

# PROC FILES
PROC_STOCK = PROC_DATA / "proc_stock.csv"
PROC_CARBON = PROC_DATA / "proc_carbon.csv"

STOCK_2000 = PROC_DATA / "stock_2000.csv"

STOCK_WORK = PROC_DATA / "stock_work.csv"
STOCK2USE = PROC_DATA / "stock2use.csv"
DIC2USE = PROC_DATA / "dic2use.json"


OUT_RENDEMENT = PROC_DATA / "rendement_v1.csv"

OUT_DF_CSV = PROC_DATA / "optimize_assets.csv"

# OUT_DIC_WEIGHT = PROC_DATA / "dic_weight.pickle"

## Utils

In [4]:
def compute_enc(weights):
    """Function that computes the ENC for given weights an a given window"""
    return 1 / sum([w**2 for w in weights])
    

def compute_ir(excess_return, track_err):
    """Function that computes the information ratio given 
    the excess return and the track_err"""
    return [ex_ret / track_err for (ex_ret, track_err) in zip(excess_return, track_err)]

def compute_di(weight, stock, vol_p):
    """Function that computes the diversification index"""
    stock_vol = stock.std() * np.sqrt(12)
    return np.dot(stock_vol, weight) / (vol_p * np.sqrt(12))

# Notebook


In [5]:
stock_df = read_file(STOCK2USE).iloc[104:-4]

return_df = pd.read_csv(OUT_DF_CSV)
return_df.index = read_file(STOCK2USE).index[104:-4]
return_df.drop(columns="Unnamed: 0", inplace=True)

stock_ret = stock_df.pct_change()

sp500 = read_file(STOCK_WORK)
bm_return = equally_weighted(sp500) 

return_sp = pd.DataFrame((sp500.mean(axis=1).pct_change())*100, columns=["mu"])
return_sp = return_sp.loc[return_df.index[0]:return_df.index[-1]]

In [7]:

metrics = ["ret", "vol", "sr", 
           "track_err", "yearly_track_error",
           "ex_ret", "enc", "encb", 
           "ir", "di"]


In [8]:
dic = dict.fromkeys(return_df.columns)
for k in dic.keys():
    dic[k] = dict.fromkeys(metrics)


In [9]:
# Poids pour chacun des rebalancement
dic_weight = {
    pf: pd.read_csv(PROC_DATA / f"weight_{pf}.csv").rename(columns={'Unnamed: 0':"quarter"}).set_index("quarter")
    for pf in dic.keys()
}

In [26]:
rf = 0.05

In [27]:
for pf in dic.keys():
        
    # Rendement annualisé
    dic[pf]["ret"] = return_df[pf].mean() * 52

    # Volatilité sur les 18 ans
    dic[pf]["vol"] = return_df[pf].std() * np.sqrt(52)
    
    # Ratio de sharp
    dic[pf]["sr"] = (dic[pf]["ret"] - rf) / dic[pf]["vol"]
    
     # Tracking error
    dic[pf]["track_err"] = []
    for i in range(len(return_df)//12): # par paquet de 3 mois
        dic[pf]["track_err"].append((return_df[pf].iloc[i*12:(i+1)*12] - return_sp["mu"].iloc[i*12:(i+1)*12]).std())
    
    # Tracking global error
    dic[pf]["yearly_track_err"] = []
    for i in range(len(return_df)//(12*4)): # par paquet de 1 an
        dic[pf]["yearly_track_err"].append((return_df[pf].iloc[i*12*4:(i+1)*12*4] - return_sp["mu"].iloc[i*12*4:(i+1)*12*4]).std())
    
    # Excess return 
    dic[pf]["ex_ret"] = []
    for i in range(len(return_df)//12): # par paquet de 3 mois
        dic[pf]["ex_ret"].append( 
            (return_df[pf].iloc[i*12:(i+1)*12].mean()) - (return_sp["mu"].iloc[i*12:(i+1)*12].mean())
        )
        
    # Yearly Excess return 
    dic[pf]["yearly_ex_ret"] = []
    for i in range(len(return_df)//(12*4)): # par paquet de 1 an
        dic[pf]["yearly_ex_ret"].append( 
            (return_df[pf].iloc[i*12*4:(i+1)*12*4].mean()) - (return_sp["mu"].iloc[i*12*4:(i+1)*12*4].mean())
        )

    # Calcul de l'ENC
    dic[pf]["enc"] = dic_weight[pf].apply(compute_enc, axis=1)
    
    # Information ratio
    dic[pf]["ir"] = compute_ir(dic[pf]["ex_ret"], dic[pf]["track_err"])
        
    # Yearly Information ratio
    dic[pf]["yearly_ir"] = compute_ir(dic[pf]["yearly_ex_ret"],
                                             dic[pf]["yearly_track_err"])
    
    # Diversification index # récupérer les poids
    sum_di = 0
    for i in range(len(return_df)//12):
          sum_di += compute_di(
            weight=dic_weight[pf].iloc[i],
            stock=stock_ret.iloc[i*12:(i+1)*12],
            vol_p=return_df[pf].iloc[i*12:(i+1)*12].std())
            
    print(pf)
    print(sum_di)
    dic[pf]["di"] = sum_di / (len(return_df)//12)
    
         

ew
1.5161281619472566
gmv
0.972526926458469
maxdecor
1.3019793327318605
maxdiv
1.1439020886895854
msr
1.042117228496666


In [29]:
dic

{'ew': {'ret': 15.46086914246365,
  'vol': 16.89877855411223,
  'sr': 0.9119516592939491,
  'track_err': [1.1829968686588939,
   1.0892322732695137,
   1.0300149369949683,
   1.0675947845438123,
   1.1061082875118267,
   0.999318143388918,
   1.7556428002255908,
   1.04767495732451,
   1.0267810381440654,
   1.4279007185784267,
   1.1749534211232746,
   1.0359679719927715,
   1.556927449441648,
   1.2136624093101513,
   0.800790996902656,
   1.4339465290650826,
   1.1994873823298764,
   0.958063023175989,
   1.4319804269286607,
   1.9056979397108424,
   1.2085487867012756,
   1.4930692562573369,
   1.6414964632873643,
   0.9735427078915739,
   1.9974057408920471,
   2.4880308096024413,
   2.0798290596596885,
   1.9379963834741978,
   2.799619362165231,
   2.378354574049002,
   3.26212165952714,
   1.6907606580564631,
   1.8271694557527227,
   1.8818142268651699,
   1.836526544311918,
   1.162439797712692,
   1.4459614459105785,
   1.397463346492769,
   1.0901508546497032,
   1.16388753

In [31]:
%store dic

Stored 'dic' (dict)
